In [16]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda, Subtract
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image


In [17]:
# Load ResNet50 model + higher level layers
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')


In [18]:
# Define Siamese Network
input_left = Input(shape=(200, 245, 3))
input_right = Input(shape=(200, 245, 3))

encoded_left = base_model(input_left)
encoded_right = base_model(input_right)

# L1 distance layer between the two encoded outputs
L1_distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([encoded_left, encoded_right])
prediction = Dense(1, activation='sigmoid')(L1_distance)

siamese_net = Model(inputs=[input_left, input_right], outputs=prediction)

optimizer = Adam(0.0001)
siamese_net.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [19]:
def load_image(img_path):
    img = image.load_img(img_path, target_size=(200, 245))
    img_array = image.img_to_array(img)
    return preprocess_input(img_array)

train_df = pd.read_csv('train.csv')

left_images = [load_image(os.path.join('train/left', fname + '.jpg')) for fname in train_df['left']]
right_images = [load_image(os.path.join('train/right', fname + '.jpg')) for fname in train_df['right']]
labels = np.ones(len(left_images))  # As they are all positive pairs

left_images = np.array(left_images)
right_images = np.array(right_images)


In [20]:
siamese_net.fit([left_images, right_images], labels, batch_size=16, epochs=2)


Epoch 1/2
125/125 [==============================] - 699s 5s/step - loss: 0.0467 - accuracy: 0.9825
Epoch 2/2
125/125 [==============================] - 623s 5s/step - loss: 0.0011 - accuracy: 1.0000


In [21]:
test_df = pd.read_csv('test_candidates.csv')
predictions = []

for index, row in test_df.iterrows():
    left_img = load_image(os.path.join('test/left', row['left'] + '.jpg'))
    left_img_batch = np.tile(np.expand_dims(left_img, axis=0), (20, 1, 1, 1))  # Replicate the left image 20 times
    
    right_imgs = [load_image(os.path.join('test/right', row[f'c{i}'] + '.jpg')) for i in range(20)]
    right_img_batch = np.array(right_imgs)  # Shape will be (20, height, width, channels)
    
    confidences = siamese_net.predict([left_img_batch, right_img_batch])
    predictions.append(confidences[:, 0])




1/1 [==============================] - 1s 1s/step


In [22]:
submission_df = pd.DataFrame(predictions, columns=[f'c{i}' for i in range(20)])
submission_df.insert(0, 'left', test_df['left'])
submission_df.to_csv('submission.csv', index=False)
